<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/%EC%9D%BC%EB%B3%84%20%EC%88%98%EC%9D%B5%EB%A5%A0%2010%25%EC%9D%B4%EC%83%81%20%EC%A2%85%EB%AA%A9%20%EC%82%AC%ED%9B%84%20%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 2.0 MB/s eta 0:00:00


In [2]:
import FinanceDataReader as fdr
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm
import time
import multiprocessing
import concurrent.futures

In [3]:
multiprocessing.cpu_count()

2

In [4]:
KS = fdr.StockListing('KOSPI') # KOSPI: 940 종목
KDQ = fdr.StockListing('KOSDAQ') # KOSDAQ: 1,597 종목

In [6]:
KS = fdr.StockListing('KOSPI') # KOSPI: 940 종목
code=KS['Code'].tolist()
code1 = [code for code in code if len(code) == 6 and code[-1] == '0']
codes = code1[:200]

In [ ]:
start_date = datetime.date(2017, 1, 1)
end_date = datetime.date(2024, 8, 16)
df1 = fdr.DataReader('KS200', start_date, end_date)
df1['Change']

In [8]:
# 날짜 범위 설정
start_date = '2017-01-01'
end_date = '2024-08-19'
start_time = time.time()

def fetch_stock_data(code):
    try:
        # 각 종목의 데이터프레임을 불러오고 'Code' 열을 추가
        df = fdr.DataReader(code, start_date, end_date)
        df['Code'] = code
        return df
    except Exception as e:
        print(f"Error fetching data for {code}: {e}")
        return pd.DataFrame()

# 병렬 처리를 위한 ThreadPoolExecutor 사용
with concurrent.futures.ThreadPoolExecutor() as executor:
    # tqdm을 사용하여 프로그래스 바 표시
    results = list(tqdm(executor.map(fetch_stock_data, codes), total=len(codes)))

# 결과 DataFrame 병합
merge_stock_df = pd.concat(results, ignore_index=False)

# 인덱스를 날짜로 설정
merge_stock_df.index = pd.to_datetime(merge_stock_df.index)

end_time = time.time()
print('--- 걸린시간: {} 초 ---'.format(end_time - start_time))

# 데이터 확인
print(merge_stock_df.head())

100%|██████████| 200/200 [00:35<00:00,  5.71it/s]

--- 걸린시간: 35.12476658821106 초 ---
             Open   High    Low  Close  Volume    Change    Code
Date                                                            
2017-01-02  35980  36240  35880  36100   93012  0.001665  005930
2017-01-03  36280  36620  36020  36480  147153  0.010526  005930
2017-01-04  36500  36520  36100  36160  159435 -0.008772  005930
2017-01-05  36060  36060  35540  35560  219349 -0.016593  005930
2017-01-06  36180  36440  36040  36200  177619  0.017998  005930


In [9]:
merge_stock_df_reset = merge_stock_df.reset_index()

# 'Change' 열만 선택하고 'Code'를 칼럼으로 설정
final_df = merge_stock_df_reset[['Date', 'Code', 'Change']]

# 피벗 테이블을 사용하여 'Date'를 인덱스로, 'Code'를 칼럼으로 설정
final_df_pivot = final_df.pivot(index='Date', columns='Code', values='Change')

In [10]:
final_df_pivot=pd.concat([final_df_pivot, df1['Change'].rename('BM')], axis=1)

In [11]:
binary_pct_change = (final_df_pivot > 0.10).astype(int)
binary_pct_change=binary_pct_change.shift(1)

In [73]:
def calculate_absolute_cumulative_returns(df, target_date, periods):
    # 기준 날짜 이후의 데이터만 필터링
    df_filtered = df[df.index >= target_date]

    # 지정한 기간만큼의 데이터 추출
    df_filtered = df_filtered.head(periods)

    # 누적 수익률 계산
    cumulative_returns = (1 + df_filtered).cumprod() - 1

    return cumulative_returns.iloc[-1] if not cumulative_returns.empty else pd.Series()

def calculate_relative_cumulative_returns(df, target_date, periods):
    # 기준 날짜 이후의 데이터만 필터링
    df_filtered = df[df.index >= target_date]

    # 벤치마크 수익률 열 추출 및 제거
    if 'BM' not in df_filtered.columns:
        raise KeyError("Column 'BM' not found in the DataFrame.")

    bm_returns = df_filtered['BM']
    df_filtered = df_filtered.drop(columns=['BM'])

    # 지정한 기간만큼의 데이터 추출
    df_filtered = df_filtered.head(periods)
    bm_returns = bm_returns.loc[df_filtered.index]

    # 누적 수익률 계산
    cumulative_returns = (1 + df_filtered).cumprod() - 1
    cumulative_bm_returns = (1 + bm_returns).cumprod() - 1

    # 상대 수익률 계산
    relative_returns = cumulative_returns.subtract(cumulative_bm_returns, axis=0)

    return relative_returns.iloc[-1] if not relative_returns.empty else pd.Series()


In [74]:
periods_list = [5, 10, 20, 30, 120]
absolute_results = []
relative_results = []

for column in binary_pct_change.columns:  # '000080', '000100' 열에 대해
    target_dates = binary_pct_change[binary_pct_change[column] == 1].index
    for target_date in target_dates:
        for periods in periods_list:
            # 절대 수익률 계산
            filtered_returns_df = final_df_pivot[[column]]
            absolute_return = calculate_absolute_cumulative_returns(filtered_returns_df, target_date, periods)
            absolute_result_entry = {
                '기준날짜': target_date,
                '기간': periods,
                '종목코드': column,
                '절대_수익률': absolute_return[column]
            }
            absolute_results.append(absolute_result_entry)

            # 상대 수익률 계산
            filtered_returns_df = final_df_pivot[['BM', column]]
            relative_return = calculate_relative_cumulative_returns(filtered_returns_df, target_date, periods)
            relative_result_entry = {
                '기준날짜': target_date,
                '기간': periods,
                '종목코드': column
            }
            if not relative_return.empty:
                relative_result_entry.update(relative_return.to_dict())
            relative_results.append(relative_result_entry)

# 결과 데이터프레임 생성
absolute_df = pd.DataFrame(absolute_results)
relative_df = pd.DataFrame(relative_results)

In [63]:
# 중복된 기간과 종목코드 조합의 평균 수익률 계산
df_agg = absolute_df.groupby(['기간', '종목코드'], as_index=False).agg({'절대_수익률': 'mean'})

# 피벗 테이블 생성
pivot_table = df_agg.pivot(index='기간', columns='종목코드', values='절대_수익률')

# 인덱스를 정렬하고 필요한 기간 추가 (30일, 120일)
periods = [5, 10, 20, 30, 120]
for period in periods:
    if period not in pivot_table.index:
        pivot_table.loc[period] = None

# 정렬 및 출력
absolute_df_pivot_table = pivot_table.sort_index()
absolute_df_pivot_table

종목코드,000080,000100,000120,000150,000240,000270,000660,000720,000810,000880,...,373220,375500,377300,383220,402340,443060,450080,454910,457190,462870
기간,,,,,,,,,,,,,,,,,,,,,
5,0.053719,-0.001102,0.051085,-0.019693,-0.050482,0.006792,0.058450,0.020960,-0.016048,-0.028004,...,-0.158055,-0.060079,-0.030177,-0.141892,0.056517,-0.063018,0.116826,0.053761,0.119154,0.227273
10,0.159091,0.037446,0.135878,-0.036961,-0.042549,0.023380,0.090216,0.131952,0.003919,0.007204,...,-0.100304,0.017175,-0.061805,-0.221622,0.093965,-0.107855,0.123021,0.006620,0.212518,-0.020376
20,0.202479,0.002915,0.176201,-0.060639,-0.064076,0.054806,0.066074,0.196430,-0.002912,0.032072,...,-0.128673,0.068684,-0.096969,-0.286486,0.126390,-0.165528,0.295109,0.230902,0.205279,-0.020376
30,0.454545,-0.015472,0.234463,-0.063479,-0.058914,0.044480,0.073698,0.122187,0.042337,0.049845,...,-0.144883,0.090127,-0.087306,-0.274324,0.056934,-0.157747,0.248680,0.187614,0.249728,-0.020376
120,0.543388,0.058820,0.479180,0.061469,0.152310,0.192051,0.040661,-0.004530,0.028207,0.264486,...,-0.211753,0.150223,-0.265748,-0.274324,0.184454,-0.203705,-0.237700,-0.020262,-0.005580,-0.020376


In [84]:
# 데이터프레임을 긴 형태로 변환 (melt)
relative_df_long = pd.melt(relative_df, id_vars=['기간', '종목코드'], var_name='종목', value_name='수익률')

# 수익률 컬럼만 숫자형으로 변환 (필요 시)
relative_df_long['수익률'] = pd.to_numeric(relative_df_long['수익률'], errors='coerce')

# 중복된 기간과 종목코드 조합의 평균 수익률 계산
relative_df_agg = relative_df_long.groupby(['기간', '종목'], as_index=False).agg({'수익률': 'mean'})

# 피벗 테이블 생성
relative_df_pivot_table = relative_df_agg.pivot(index='기간', columns='종목', values='수익률')

# 필요한 기간 추가 (5, 10, 20, 30, 120일)
periods = [5, 10, 20, 30, 120]
for period in periods:
    if period not in relative_df_pivot_table.index:
        relative_df_pivot_table.loc[period] = np.nan

# NaN 열 제거
relative_df_pivot_table = relative_df_pivot_table.dropna(axis=1, how='all')

# 정렬 및 출력
relative_df_pivot_table = relative_df_pivot_table.sort_index()


In [87]:
relative_df_pivot_table.T.describe()

기간,5,10,20,30,120
count,187.000000,186.000000,184.000000,183.000000,181.000000
mean,0.003184,0.009588,0.008798,0.015258,0.017827
std,0.048154,0.063074,0.085899,0.105146,0.264957
min,-0.125510,-0.204591,-0.206992,-0.255825,-0.560715
25%,-0.020570,-0.024956,-0.041664,-0.051977,-0.145325
50%,0.001160,0.008588,0.006805,0.001093,-0.007220
75%,0.024838,0.046923,0.055946,0.066016,0.116625
max,0.182522,0.226672,0.347398,0.343626,1.070582


In [88]:
stats=relative_df_pivot_table.T.describe()
stats.loc['mean'] / stats.loc['std']

,0
기간,
5,0.066126
10,0.152010
20,0.102423
30,0.145111
120,0.067282


In [89]:
abs_stats=absolute_df_pivot_table.T.describe()
abs_stats.loc['mean'] / abs_stats.loc['std']

,0
기간,
5,0.267399
10,0.450151
20,0.494558
30,0.476610
120,0.601794


In [90]:
abs_stats

기간,5,10,20,30,120
count,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.014502,0.029991,0.045669,0.054889,0.157190
std,0.054232,0.066625,0.092344,0.115165,0.261203
min,-0.158055,-0.221622,-0.286486,-0.274324,-0.365462
25%,-0.010634,-0.002108,-0.012141,-0.016129,-0.003140
50%,0.009014,0.025478,0.038803,0.044480,0.124648
75%,0.039443,0.057081,0.087167,0.105159,0.259143
max,0.227273,0.262247,0.314685,0.534965,1.137133
